In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/test.csv')
submit = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/sample_submission.csv')

In [ ]:
train.shape

In [ ]:
train.head()

> We have to find congestion so let us have look on distribution of congestion

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.displot(train['congestion'], kde = True)

**Data is distributed normally even we can check with calculating mean, median, mode**

In [ ]:
print(train['congestion'].mean())
print(train['congestion'].median())
print(train['congestion'].mode()[0])

In [ ]:
sns.violinplot(train['congestion'])

**Lets combine test and train**

In [ ]:
target = train["congestion"]

train.drop(["congestion"], axis=1, inplace=True)
train

In [ ]:
total = train.append(test)
total

In [ ]:
total['time'] = pd.to_datetime(total['time'], format="%Y-%m-%d %H:%M:%S")
total

In [ ]:
total['month'] = pd.DatetimeIndex(total['time']).month
total

In [ ]:
total["weekday"] = total['time'].dt.dayofweek
total

In [ ]:
sns.displot(total['weekday'])

In [ ]:
total['hour'] = pd.to_datetime(total['time'], format='%Y-%m-%d %H:%M:%S').dt.hour
total

In [ ]:
total['direction'].replace({'EB':1, 'NB':2, 'SB':3, 'WB':4, 'NE':5, 'SW':6, 'NW':7,'SE':8}, inplace=True)
total

In [ ]:
features = ['x', 'y', 'direction', 'weekday', 'month', 'hour']
total[features]

In [ ]:
y = target
X = total[features][: len(train)]
X_test = total[features][len(train) :]



In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=42, stratify=y, shuffle=True)
X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=42).fit(X, y)
print(model.score(X, y))

In [ ]:
preds = model.predict(X_test)
preds = preds.astype(int)
preds[preds < 0] = 0
preds

In [ ]:
sns.displot(preds, kde = True)

***Preds and train match each other***

In [ ]:
print(preds.mean())


In [ ]:
submit.congestion = preds
submit.to_csv('submission.csv', index=False)
submission = pd.read_csv("submission.csv")
submission.head()